This notebook only works for our model so far, and runs pointwise attack. Will make pointwise script later

Some prelimenary results on MNIST 100 (using only 100 queries when doing adversarial training):

Results are for networks trained wit Beta=1

og_net: 11
trunc : 12
clip:   15
lin:    13

There is some variance



CIFAR

og_net (no adv): 2
simple (no adv): 7
simple (adv):    11

In [ ]:
# torch imports
import torch
# other modules
import foolbox 
import argparse
import numpy as np 
from tqdm import trange
import pickle
# PATHING
import os
import sys
import json
root = os.path.abspath(os.curdir)
sys.path.append(root)


root = os.path.abspath('/home/mbeliaev/home/code/robust-l0/')
sys.path.append(root)

from utils.models import load_net
from utils.helpers import *
from utils.attack import *
# from abs_models import models as mz        # model zoo
# from abs_models import utils as u







In [ ]:
device = 'cuda:3'
bs = 25
mnist_data = prep_data(root,bs,'MNIST')
cifar_data = prep_data(root,bs,'CIFAR')

In [ ]:
def run_pw_attack(model,data_x,data_y,num_iters,device):
    '''
    Runs pointwise attack on model using x,y
    NOTE: assumes x,y is list of tensors 
    '''
    fmodel = foolbox.models.PyTorchModel(model=model,
                                        bounds=(0,1),
                                        num_classes=10,
                                        channel_axis=1,
                                        device=device)

    attack = foolbox.attacks.PointwiseAttack(model=fmodel,
                                            distance=foolbox.distances.L0)
    # run attack
    advs = []
    advs_dist = []
    # for batch in trange(num_batches):
    for batch in trange(len(data_x)):
        x, y = data_x[batch].numpy(), data_y[batch].numpy()
        bs = x.shape[0]
        # final_l0 = np.ones(bs)*100000
        # final_x = x.copy() # numpy form
        adversarial_dists = []
        adversarials = []
        for _ in range(num_iters):
            adv_x = attack(x, y, unpack=False)
            distance = [x.distance.value for x in adv_x]
            original_class = [x.original_class for x in adv_x]
            output = [x.output for x in adv_x]
            adversarial_dists.append(distance)
            adversarials.append({'distance': distance,'label': original_class, 'scores':  output})

        # find min
        adversarial_dists = np.asarray(adversarial_dists)
        mindex = np.argmin(adversarial_dists,axis=0)
        min_dists = [adversarial_dists[i_mindex, i] for i,i_mindex in enumerate(mindex)]
        min_dists = np.asarray(min_dists)

        min_advs = {'distance': [], 'label': [], 'scores': []}
        for i, i_mindex in enumerate(mindex):
            for key in min_advs.keys():
                min_advs[key].append(adversarials[i_mindex][key][i])
        
        # save
        advs_dist.append(min_dists)
        advs.append(min_advs)

    advs_dist = np.asarray(advs_dist)
    print('Final median l0 distance: ',np.median(advs_dist))

    return advs, advs_dist



In [ ]:
CIFAR_paths = {'og no adv': '/home/mbeliaev/home/code/robust-l0/new_trained/final_no/CIFAR/1/VGG16/og/k_0/adv_12/0/',
                'simple 12 no adv': '/home/mbeliaev/home/code/robust-l0/new_trained/final_no/CIFAR/1/VGG16/simple/k_12/adv_12/0/',
                'simple 100_no_adv': '/home/mbeliaev/home/code/robust-l0/new_trained/largek/CIFAR/1/VGG16/simple/k_100/adv_12/0/'}

total_samples = 25
num_iters = 2
for key in CIFAR_paths.keys():
    print('-------'*8)
    print(key)
    net_path = CIFAR_paths[key]
    print(net_path)
    model = load_net(net_path, device, input_shape=cifar_data['x_test'][0].shape)
    model.to(device)
    acc = evaluate(model,cifar_data['x_test'],cifar_data['y_test'],device)
    print('Clean accuracy: ',acc)

    # run pointwise attack
    advs, advs_dist = run_pw_attack(model=model, 
                                data_x=cifar_data['x_test'][0:total_samples//bs],
                                data_y=cifar_data['y_test'][0:total_samples//bs],
                                num_iters=num_iters,
                                device=device)


    print(advs_dist)
    

In [ ]:
# CIFAR_paths = {'og no adv': '/home/mbeliaev/home/code/robust-l0/new_trained/final_no/CIFAR/1/VGG16/og/k_0/adv_12/0/',
                # 'simple 12 no adv': '/home/mbeliaev/home/code/robust-l0/new_trained/final_no/CIFAR/1/VGG16/simple/k_12/adv_12/0/',
CIFAR_paths = {'simple 200_no_adv': '/home/mbeliaev/home/code/robust-l0/new_trained/largek/CIFAR/1/VGG16/simple/k_200/adv_12/0/'}

total_samples = 25
num_iters = 2
for key in CIFAR_paths.keys():
    print('-------'*8)
    print(key)
    net_path = CIFAR_paths[key]
    print(net_path)
    model = load_net(net_path, device, input_shape=cifar_data['x_test'][0].shape)
    model.to(device)
    acc = evaluate(model,cifar_data['x_test'],cifar_data['y_test'],device)
    print('Clean accuracy: ',acc)

    # run pointwise attack
    advs, advs_dist = run_pw_attack(model=model, 
                                data_x=cifar_data['x_test'][0:total_samples//bs],
                                data_y=cifar_data['y_test'][0:total_samples//bs],
                                num_iters=num_iters,
                                device=device)


    print(advs_dist)
    

In [ ]:
MNIST_paths = {'og no adv': '/home/mbeliaev/home/code/robust-l0/new_trained/final_no/MNIST/1/cnn_small/og/k_0/adv_12/0/',
                'simple 12 no adv': '/home/mbeliaev/home/code/robust-l0/new_trained/final_no/MNIST/1/cnn_small/simple/k_12/adv_12/0/',
                'simple 100_no_adv': '/home/mbeliaev/home/code/robust-l0/new_trained/largek/MNIST/1/cnn_small/simple/k_100/adv_12/0/' }

total_samples = 25
num_iters = 2
for key in MNIST_paths.keys():
    print('-------'*8)
    print(key)
    net_path = MNIST_paths[key]
    print(net_path)
    model = load_net(net_path, device, input_shape=mnist_data['x_test'][0].shape)
    model.to(device)
    acc = evaluate(model,mnist_data['x_test'],mnist_data['y_test'],device)
    print('Clean accuracy: ',acc)

    # run pointwise attack
    advs, advs_dist = run_pw_attack(model=model, 
                                data_x=mnist_data['x_test'][0:total_samples//bs],
                                data_y=mnist_data['y_test'][0:total_samples//bs],
                                num_iters=num_iters,
                                device=device)


    print(advs_dist)
    